In [ ]:
# %%
import sys, os

try:
    # ✅ Running from a Python script (.py file)
    base_path = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

SRC_PATH = os.path.join(base_path)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

In [ ]:
# agents/restaurant/slot_collector_agent.py

from langchain_together import ChatTogether
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain.tools import StructuredTool
from schema.slot_schema import SlotCollectorInput, SlotCollectorOutput

# Dummy logic (LLM will actually parse)
def extract_user_info(query: str) -> dict:
    return {
        "name": "Rahul",
        "phone": "9876543210",
        "table_number": 4,
        "number_of_people": 2
    }

slot_collector_tool = StructuredTool.from_function(
    name="slot_collector_tool",
    description="Extracts name, phone, table number, and party size from the user's input.",
    func=extract_user_info,
    args_schema=SlotCollectorInput,
    return_schema=SlotCollectorOutput
)

# Agent Setup
llm = ChatTogether(model="mistralai/Mistral-7B-Instruct-v0.2", api_key=os.getenv("together_ai_api_key"), temperature=0)

prompt_template = PromptTemplate.from_template(
"""
You're a helpful slot-filler agent for a restaurant. You need to extract the user's name, phone, table number, and how many people are dining.

Always call `slot_collector_tool` with the query input.

Example:
{{"action": "slot_collector_tool", "action_input": {{"query": "<user message>"}}}}

Begin!

Human: {input}
"""
)

slot_agent = create_structured_chat_agent(
    llm=llm,
    tools=[slot_collector_tool],
    prompt=prompt_template
)

slot_executor = AgentExecutor(agent=slot_agent, tools=[slot_collector_tool], verbose=True)
